In [3]:
import os, sys
import shutil
import time
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go



# include the folder
sys.path.append(os.path.join(os.getcwd(), os.path.pardir))

import navier_stokes_2d
import post_prosessing as pp

%load_ext autoreload
%autoreload 2

%matplotlib qt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
output_root_folder = r'/home/youngsuklee/SimulationData/NS2D'

# 

# Kelvin Helmhotz simulation

In [18]:
# prepare the data io object
dio = pp.DataIO(os.path.join(output_root_folder, 'KH2D'))
dio.clean_all()

pi2 = 2.0 * np.pi
Mx = 256
My = Mx
Mx_da = None
My_da = Mx_da
Re = 10000
ns2d = navier_stokes_2d.NS2dSpectral(pi2, pi2, Mx, My, 1/Re, 1, Mx_da=Mx_da, My_da=My_da, FFTW_threads=6)

# initialise
mtx_vor = navier_stokes_2d.init_kevin_helmoltz_vorticity_periodic(Mx=Mx, My=My)
mtx_vor_k =  ns2d.fft2d(mtx_vor)

T = 1000
t = 0.0
for n in range(T):
    if n % 20 == 0:
        print('\r time step (n): ', str(n), end='')
        ns2d.update_dt_step_k(mtx_vor_k)
        mtx_vor = ns2d.ifft2d(mtx_vor_k)
        dio.save_data(str(T+n), mtx_vor, {'t': t})
    mtx_vor_k = ns2d.march_forward_k(mtx_vor_k)
    t += ns2d.dt_step

 time step (n):  0
*** time step updated:  0.06392079769627605  from  inf

 time step (n):  60
*** time step updated:  0.04117738379032788  from  0.06392079769627605

 time step (n):  980

In [6]:
pi2 = 2.0 * np.pi
Mx = 256
My = Mx
Mx_da = None
My_da = Mx_da
Re = 10000
ns2d = navier_stokes_2d.NS2dSpectral(pi2, pi2, Mx, My, 1/Re, 1, Mx_da=Mx_da, My_da=My_da, FFTW_threads=6)

In [14]:
dio = pp.DataIO(os.path.join(output_root_folder, 'KH2D'))
mtx_vor, kw_atts = dio.read_data(str(1200))
mtx_psi = ns2d.get_psi(mtx_vor)

mtx_u, mtx_v = ns2d.get_uv_from_psi(mtx_psi)

plt.figure(figsize=(10,10))
plt.streamplot(ns2d.mtx_x, ns2d.mtx_y, mtx_u, mtx_v, linewidth=0.5)
pp.make_image_2d(mtx_vor, ns2d.mtx_x, ns2d.mtx_y)
#plt.colorbar()
plt.savefig(os.path.join(dio.fig_folder, 'KH.png'))
plt.show()

# Turbulence

In [38]:
# prepare the data io object
dio = pp.DataIO(os.path.join(output_root_folder, 'Turb2D'))
dio.clean_all()

pi2 = 2.0 * np.pi
Mx = 256
My = Mx
Mx_da = None
My_da = Mx_da
Re = 10000
ns2d = navier_stokes_2d.NS2dSpectral(pi2, pi2, Mx, My, 1/Re, 1, Mx_da=Mx_da, My_da=My_da, FFTW_threads=6)

# initialise
mtx_vor = navier_stokes_2d.init_random_periodic(Mx=Mx, My=My)
mtx_vor_k =  ns2d.fft2d(mtx_vor)
mtx_vor_k /= ns2d.get_energy(mtx_vor_k)

In [83]:
512 * 2 / 3

341.3333333333333

In [150]:
# prepare the data io object
dio = pp.DataIO(os.path.join(output_root_folder, 'Turb2D_E1'))
dio.clean_all()

pi2 = 2.0 * np.pi
Mx = 340
My = Mx
Mx_da = 512
My_da = Mx_da
Re = 10000
ns2d = navier_stokes_2d.NS2dSpectral(pi2, pi2, Mx, My, 1/Re, 1, Mx_da=Mx_da, My_da=My_da, FFTW_threads=6)

# initialise
mtx_vor = navier_stokes_2d.init_random_periodic(Mx=Mx, My=My, k0=50)
mtx_vor_k =  ns2d.fft2d(mtx_vor)
mtx_vor_k /= np.sqrt(ns2d.get_energy(mtx_vor_k)) # make it a unit energy
mtx_vor_k_0 = mtx_vor_k.copy()

T = 10000
t = 0.0
for n in range(T):
    if n % 200 == 0:
        print('\r time step (n): ', str(n), end='')
        ns2d.update_dt_step_k(mtx_vor_k)
        mtx_vor = ns2d.ifft2d(mtx_vor_k)
        dio.save_data(str(T+n), mtx_vor, {'t': t})
    mtx_vor_k = ns2d.march_forward_k(mtx_vor_k)
    t += ns2d.dt_step

 time step (n):  0
*** time step updated:  0.018398253140038522  from  inf

 time step (n):  400
*** time step updated:  0.05125305397466251  from  0.018398253140038522

 time step (n):  1200
*** time step updated:  0.11429002581396211  from  0.05125305397466251

 time step (n):  6000
*** time step updated:  0.2352854252232887  from  0.11429002581396211

 time step (n):  9800

In [20]:
dio = pp.DataIO(os.path.join(output_root_folder, 'Turb2D_E1'))

pi2 = 2.0 * np.pi
Mx = 340
My = Mx
Mx_da = 512
My_da = Mx_da
Re = 10000
ns2d = navier_stokes_2d.NS2dSpectral(pi2, pi2, Mx, My, 1/Re, 1, Mx_da=Mx_da, My_da=My_da, FFTW_threads=6)

mtx_vor, kw_atts = dio.read_data(str(10000 + 1800))
mtx_vor_k = ns2d.fft2d(mtx_vor)

plt.figure(figsize=(10,10))
pp.make_image_2d(mtx_vor, ns2d.mtx_x, ns2d.mtx_y, 'RdBu_r')
plt.savefig(os.path.join(dio.fig_folder, 'Turb2D.png'))
plt.show()

In [147]:
l_t = [0, 9800]
for t in l_t:
    mtx_vor, kw_atts = dio.read_data(str(T + t))
    mtx_vor_k = ns2d.fft2d(mtx_vor)
    print('energy', ns2d.get_energy(mtx_vor_k))

energy nan
energy nan


In [137]:
ns2d.get_energy(mtx_vor_k_0)

2288.246040393053

In [144]:
mtx_vor = navier_stokes_2d.init_random_periodic(Mx=Mx, My=My, k0=50)
mtx_vor_k =  ns2d.fft2d(mtx_vor)
E = ns2d.get_energy(mtx_vor_k)
mtx_vor_k /= np.sqrt(E)
mtx_vor_k_0 = mtx_vor_k.copy()